## Importing packages:

In [1]:
from __future__ import print_function, division  # for compatibility with python 3.x
import warnings
warnings.filterwarnings('ignore')  # don't print out warnings

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
#import seaborn as sns

In [3]:
from statsmodels.multivariate.manova import MANOVA
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [4]:
# File from processed vibration data notebook (raw-data-processing(sabeer,vishal 12Aug2020).ipynb)
df = pd.read_excel(r'C:\Users\husssabe\Desktop\Projects\Maintenance 4.0\TestLoop\ProcessedDataFeaturesandFFT.xlsx',header=0)
#df=pd.read_excel(r'C:\_WorkDir\PythonProj\IT Analysis\Testloop\VibExtraction\Output\ProcessedVibMatrices.xlsx')

In [5]:
df.columns

Index(['Unnamed: 0', 'Asset_Id', 'Axis', 'Data_pull_time', 'Defect_Introduced',
       'Mac_Id', 'Sensor_Loc', 'crest', 'freqAmpliKurt', 'freqAmpliMax',
       'freqAmpliMean', 'freqAmpliVar', 'freqAmpliskw', 'kurt', 'max_1',
       'max_2', 'max_3', 'max_4', 'max_5', 'max_6', 'max_7', 'max_8', 'mean',
       'mean_1', 'mean_2', 'mean_3', 'mean_4', 'mean_5', 'mean_6', 'mean_7',
       'mean_8', 'peak-peak', 'rms', 'skewness', 'var_1', 'var_2', 'var_3',
       'var_4', 'var_5', 'var_6', 'var_7', 'var_8', 'variance'],
      dtype='object')

In [6]:
df=df.drop(columns=['Unnamed: 0'],axis=1)


In [7]:
df=df.reset_index()

In [8]:
for col in df.columns:
    if df[col].dtype==object:
        df[col]=df[col].astype('category')
        
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   index              2075 non-null   int64         
 1   Asset_Id           2075 non-null   category      
 2   Axis               2075 non-null   category      
 3   Data_pull_time     2075 non-null   datetime64[ns]
 4   Defect_Introduced  2075 non-null   category      
 5   Mac_Id             2075 non-null   category      
 6   Sensor_Loc         2075 non-null   category      
 7   crest              2075 non-null   float64       
 8   freqAmpliKurt      2075 non-null   float64       
 9   freqAmpliMax       2075 non-null   float64       
 10  freqAmpliMean      2075 non-null   float64       
 11  freqAmpliVar       2075 non-null   float64       
 12  freqAmpliskw       2075 non-null   float64       
 13  kurt               2075 non-null   float64       
 14  max_1   

In [9]:
df.Defect_Introduced.value_counts()

No Defect_No Load                                    616
No Defect_Empty Tray                                 383
No Defect_with no tray                               278
Decline-Non Drive Side Tesnion unit faulty bea...    173
non drive side faulty bearing was introduced_w...    160
slight wobbling in the motor_Empty Tray              120
Slight Wobling in the motor                          117
Non Drive side faulty bearing was introduced -...    114
Belt showing signs of wear and tear, slight wo...     54
slight wobbling in the motor_with no tray             45
slight wobbling in the motor_No Load                  15
Name: Defect_Introduced, dtype: int64

In [10]:
df.Asset_Id.value_counts()

101.CA.DTL.01.1-050    637
101.CA.DTL.01.1-200    450
101.CA.DTL.01.1-210    351
101.CA.DTL.01.1-040    207
101.CA.DTL.01.1-020    173
101.CA.DTL.01.1-330    160
101.CA.DTL.01.1-310     97
Name: Asset_Id, dtype: int64

In [11]:
df.Sensor_Loc.value_counts()

MET_EDT    949
MET_TST    471
MET-EDT    333
MET_TUR    124
MET_TUL     69
MET_DSF     36
MET_CFL     33
MET_DUR     32
MET_DUL     18
MET_CFR     10
Name: Sensor_Loc, dtype: int64

In [13]:
dfedt=df[df.Sensor_Loc.isin(['MET_EDT','MET_EDT'])]
dfedt.head()
dfedt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 949 entries, 223 to 2074
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   index              949 non-null    int64         
 1   Asset_Id           949 non-null    category      
 2   Axis               949 non-null    category      
 3   Data_pull_time     949 non-null    datetime64[ns]
 4   Defect_Introduced  949 non-null    category      
 5   Mac_Id             949 non-null    category      
 6   Sensor_Loc         949 non-null    category      
 7   crest              949 non-null    float64       
 8   freqAmpliKurt      949 non-null    float64       
 9   freqAmpliMax       949 non-null    float64       
 10  freqAmpliMean      949 non-null    float64       
 11  freqAmpliVar       949 non-null    float64       
 12  freqAmpliskw       949 non-null    float64       
 13  kurt               949 non-null    float64       
 14  max_1  

In [14]:
#dfedtc=df.loc[df.Asset_Id.isin(['101.CA.DTL.01.1-020','101.CA.DTL.01.1-200'])]
#dfedtc=df.loc[df.Asset_Id.isin(['101.CA.DTL.01.1-020'])]
#dfedtc=df.loc[df.Asset_Id=='101.CA.DTL.01.1-020']
#dfedtc=df.query('Asset_Id=="101.CA.DTL.01.1-020"')
#dfedtc.info()

In [15]:
dfedt.head()

,index,Asset_Id,Axis,Data_pull_time,Defect_Introduced,Mac_Id,Sensor_Loc,crest,freqAmpliKurt,freqAmpliMax,...,skewness,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,variance
223,223,101.CA.DTL.01.1-050,X,2020-07-21 12:37:43,No Defect_Empty Tray,29:9c:fe:16,MET_EDT,17.692794,13.104953,102.389972,...,0.260243,458.382383,130.516519,148.652834,85.562790,78.334326,49.546852,104.145603,49.245669,0.054294
224,224,101.CA.DTL.01.1-050,X,2020-07-21 13:32:32,No Defect_Empty Tray,29:9c:fe:16,MET_EDT,6.550334,36.088744,125.279978,...,0.013241,104.604739,177.722643,375.606772,114.221710,31.538522,38.286608,9.358293,158.447675,0.038136
225,225,101.CA.DTL.01.1-050,X,2020-07-21 13:49:25,No Defect_Empty Tray,29:9c:fe:16,MET_EDT,5.147500,90.016390,214.603791,...,-0.015044,76.951279,178.837577,515.690654,121.555973,48.554712,34.877534,10.868388,156.264216,0.041601
226,226,101.CA.DTL.01.1-050,X,2020-07-21 14:15:48,No Defect_Empty Tray,29:9c:fe:16,MET_EDT,6.142973,38.604170,188.883617,...,-0.114612,221.827532,148.266681,1264.315215,241.608089,41.393605,69.416401,20.307308,99.034283,0.076713
227,227,101.CA.DTL.01.1-050,X,2020-07-21 15:06:27,No Defect_Empty Tray,29:9c:fe:16,MET_EDT,4.920943,30.354636,182.508059,...,0.054166,238.830572,160.239836,1044.433799,290.990351,37.595005,45.592705,14.799536,82.720978,0.074345


In [16]:
dfedt[dfedt.Asset_Id=='101.CA.DTL.01.1-330']

,index,Asset_Id,Axis,Data_pull_time,Defect_Introduced,Mac_Id,Sensor_Loc,crest,freqAmpliKurt,freqAmpliMax,...,skewness,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,variance


In [17]:
dfedt.Asset_Id.value_counts()

101.CA.DTL.01.1-200    312
101.CA.DTL.01.1-050    291
101.CA.DTL.01.1-210    135
101.CA.DTL.01.1-040    114
101.CA.DTL.01.1-310     97
101.CA.DTL.01.1-330      0
101.CA.DTL.01.1-020      0
Name: Asset_Id, dtype: int64

In [18]:
dfedt.Asset_Id.unique()

['101.CA.DTL.01.1-050', '101.CA.DTL.01.1-210', '101.CA.DTL.01.1-200', '101.CA.DTL.01.1-310', '101.CA.DTL.01.1-040']
Categories (5, object): ['101.CA.DTL.01.1-050', '101.CA.DTL.01.1-210', '101.CA.DTL.01.1-200', '101.CA.DTL.01.1-310', '101.CA.DTL.01.1-040']

In [19]:
test21=dfedt[dfedt.Asset_Id.isin(['101.CA.DTL.01.1-050', '101.CA.DTL.01.1-210', '101.CA.DTL.01.1-200', '101.CA.DTL.01.1-310', '101.CA.DTL.01.1-040'])]
test21.Asset_Id.value_counts()

101.CA.DTL.01.1-200    312
101.CA.DTL.01.1-050    291
101.CA.DTL.01.1-210    135
101.CA.DTL.01.1-040    114
101.CA.DTL.01.1-310     97
101.CA.DTL.01.1-330      0
101.CA.DTL.01.1-020      0
Name: Asset_Id, dtype: int64

In [20]:
dfedt.Defect_Introduced.value_counts()

No Defect_No Load                                    393
No Defect_Empty Tray                                 210
Non Drive side faulty bearing was introduced -...    114
No Defect_with no tray                                97
slight wobbling in the motor_Empty Tray               66
Belt showing signs of wear and tear, slight wo...     54
slight wobbling in the motor_No Load                  15
slight wobbling in the motor_with no tray              0
non drive side faulty bearing was introduced_w...      0
Slight Wobling in the motor                            0
Decline-Non Drive Side Tesnion unit faulty bea...      0
Name: Defect_Introduced, dtype: int64

In [ ]:
# Writing to Excel

writer= pd.ExcelWriter('C:\Python\WorkDir\Maintenance 4.0\TestLoop\VibExtraction\Output\dfedt.xlsx',
                       engine='xlsxwriter', datetime_format='yyyy-mm-dd HH:mm:ss')

dfedt.to_excel(writer, sheet_name='test', 
                                 index = False,  header=True)
# Close the Pandas Excel writer and output the Excel file.
writer.save()
# Remove DefectC and give

In [ ]:
x=np.where(dfedt.Defect_Introduced.value_counts() >0)
x

In [ ]:
dfedt.boxplot(column='mean',by='Defect_Introduced',figsize=(12,6))

In [ ]:
pd.pivot_table(dfedt, index = 'Defect_Introduced',columns= 'Asset_Id',values='rms',aggfunc = 'count')

In [ ]:
EDTgrouped=dfedt.loc[:,"Defect_Introduced":"variance"]
EDTgrouped=EDTgrouped.drop(columns=['Mac_Id','Sensor_Loc'])
EDTgrouped.head()

In [ ]:
EDTgrouped.info()

In [ ]:
X = EDTgrouped.loc[:,"crest":]  # independent variables data
y = EDTgrouped.Defect_Introduced  # dependednt variable data
EDTgrouped.head()

### # Mean & std

In [ ]:
X.apply(np.mean)

In [ ]:
# Std Deviation
X.apply(np.std)

In [ ]:
#Mean and Std deviation by Gp
def printMeanAndSdByGroup(variables, groupvariable):
    data_groupby = variables.groupby(groupvariable)
    print("## Means:")
    display(data_groupby.apply(np.mean))
    print("\n## Standard deviations:")
    display(data_groupby.apply(np.std))
    print("\n## Sample sizes:")
    display(pd.DataFrame(data_groupby.apply(len)))

In [ ]:
printMeanAndSdByGroup(X, y)

### Standardisation

In [ ]:
standardisedX = scale(X)
standardisedX = pd.DataFrame(standardisedX, index=X.index, columns=X.columns)

In [ ]:
standardisedX.apply(np.mean)

In [ ]:
standardisedX.apply(np.std)

### PCA

In [ ]:
pca = PCA().fit(standardisedX)

In [ ]:

def pca_summary(pca, standardised_data, out=True):
    names = ["PC"+str(i) for i in range(1, len(pca.explained_variance_ratio_)+1)]
    a = list(np.std(pca.transform(standardised_data), axis=0))
    b = list(pca.explained_variance_ratio_)
    c = [np.sum(pca.explained_variance_ratio_[:i]) for i in range(1, len(pca.explained_variance_ratio_)+1)]
    columns = pd.MultiIndex.from_tuples([("sdev", "Standard deviation"), ("varprop", "Proportion of Variance"), ("cumprop", "Cumulative Proportion")])
    summary = pd.DataFrame(zip(a, b, c), index=names, columns=columns)
    if out:
        print("Importance of components:")
        display(summary)
    return summary

In [ ]:
summary = pca_summary(pca, standardisedX)

In [ ]:
def screeplot(pca, standardised_values):
    y = np.std(pca.transform(standardised_values), axis=0)**2
    x = np.arange(len(y)) + 1
    plt.plot(x, y, "o-")
    plt.xticks(x, ["Comp."+str(i) for i in x], rotation=60)
    plt.ylabel("Variance")
    plt.show()

screeplot(pca, standardisedX)

In [ ]:
# Using Kaiser’s criterion: that we should only retain principal components for which the variance is above 1
summary.sdev**2

In [ ]:
def pca_scatter(pca, standardised_values, classifs):
    foo = pca.transform(standardised_values)
    bar = pd.DataFrame(zip(foo[:, 0], foo[:, 1], classifs), columns=["PC1", "PC2", "Class"])
    sns.lmplot("PC1", "PC2", bar, hue="Class", fit_reg=False)

pca_scatter(pca, standardisedX, y)

### LDA

In [ ]:
lda = LinearDiscriminantAnalysis().fit(X, y)

In [ ]:
def pretty_scalings(lda, X, out=False):
    ret = pd.DataFrame(lda.scalings_, index=X.columns, columns=["LD"+str(i+1) for i in range(lda.scalings_.shape[1])])
    if out:
        print("Coefficients of linear discriminants:")
        display(ret)
    return ret

pretty_scalings_ = pretty_scalings(lda, X, out=True)

In [ ]:
def rpredict(lda, X, y, out=False):
    ret = {"class": lda.predict(X),
           "posterior": pd.DataFrame(lda.predict_proba(X), columns=lda.classes_)}
    ret["x"] = pd.DataFrame(lda.fit_transform(X, y))
    ret["x"].columns = ["LD"+str(i+1) for i in range(ret["x"].shape[1])]
    if out:
        print("class")
        print(ret["class"])
        print()
        print("posterior")
        print(ret["posterior"])
        print()
        print("x")
        print(ret["x"])
    return ret

lda_values = rpredict(lda, standardisedX, y, True)

In [ ]:
printMeanAndSdByGroup(lda_values["x"], y);